# 0. Why do people use GNNs?
Many real-world datasets have rich relational structures (e.g., social networks, citation networks, molecular graphs, knowledge graphs). 
Traditional fully-connected or convolutional networks aren’t designed to this graph structure. 
GNNs fill this gap by explicitly using the graph’s adjacency information during the forward pass.

# 1. Introduction to Graph Neural Networks

In this tutorial, we will explore **Graph Neural Networks (GNNs)**, 
which are specialized neural networks designed to handle data represented as graphs (nodes + edges). 
We will cover:

1. The high-level idea behind GNNs (Message Passing).
2. The **Graph Convolutional Network (GCN)** architecture.
3. A simple example of GCN-based node classification.
4. An introduction to **Relational GCN (R-GCN)** for multi-relational data.

## 2. Setup

Below, we load common libraries needed for our experiments. Make sure you have installed:
- `torch` (PyTorch)
- `numpy`

You can install them via `pip install torch numpy`.

*(If you have `torch-geometric`, we’ll discuss how to use it in an optional section. But it’s not required for the core tutorial.)*

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

print("PyTorch version:", torch.__version__)

PyTorch version: 

# 3. Graph Notation & Message Passing

### 3.1 Graph Notation
A graph **G = (V, E)** has:

- **V**: a set of nodes (or vertices), $|V| = N$.
- **E**: a set of edges. Each edge is a connection $(i, j)$ between two nodes.

We often store this information in an **adjacency matrix** $A \in \{0,1\}^{N \times N}$, where:

$$
A_{ij} = \begin{cases}
1 & \text{if there is an edge between node } i \text{ and node } j, \\
0 & \text{otherwise}.
\end{cases}
$$

We also assume each node $i$ has a feature vector $x_i \in \mathbb{R}^d$. 
Collectively, we can store them in a matrix $X \in \mathbb{R}^{N \times d}$.

### 3.2 Message Passing Paradigm

The core idea behind GNNs is **message passing**:

1. Each node begins with an initial representation (often just its input features): $h_i^{(0)} = x_i$.
2. At each layer (or step) $k$, every node aggregates information from its neighbors to form a new representation $h_i^{(k)}$.
3. After multiple layers, each node’s representation captures information from a broader neighborhood in the graph.

The typical update rule is:

$$
h_i^{(k)} = \text{UPDATE}\Bigl(h_i^{(k-1)}, \text{AGGREGATE}\bigl(\{h_j^{(k-1)} : j \in \mathcal{N}(i)\}\bigr)\Bigr),
$$

where $\mathcal{N}(i)$ is the set of neighbors of $i$. 
Different GNN variants define different **AGGREGATE** and **UPDATE** functions.

# 4. Graph Convolutional Network (GCN)

One of the earliest and most widely used GNN variants is the **Graph Convolutional Network (GCN)** by Kipf & Welling (ICLR 2017). The layer update is typically written in matrix form:

$$
H^{(k+1)} = \sigma\left(\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H^{(k)} W^{(k)}\right),
$$

where:

- $\tilde{A} = A + I$ (we add self-connections along the diagonal).
- $\tilde{D}$ is the diagonal degree matrix of $\tilde{A}$, i.e., $\tilde{D}_{ii} = \sum_j \tilde{A}_{ij}$.
- $H^{(k)} \in \mathbb{R}^{N \times d_k}$ is the matrix of node embeddings in the $k$-th layer.
- $W^{(k)} \in \mathbb{R}^{d_k \times d_{k+1}}$ is a trainable weight matrix.
- $\sigma$ is a non-linear activation (e.g., ReLU).

Conceptually, **GCN** normalizes neighbor features, applies a linear transform, and then uses a non-linearity.

## 4.1 Minimal GCN Implementation

Below, we define two classes:

- **`SimpleGCNLayer`**: a single GCN layer that computes $\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} X W$.
- **`SimpleGCN`**: a 2-layer GCN network (just for illustration).

In [2]:
class SimpleGCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(SimpleGCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x, adj):
        """x: (N, d_in), adj: (N, N) adjacency with self-loops added"""
        # Compute the degree of each node
        degree = torch.sum(adj, dim=1, keepdim=True)
        degree_inv_sqrt = torch.pow(degree, -0.5)
        degree_inv_sqrt[torch.isinf(degree_inv_sqrt)] = 0.0  # prevent inf

        # Normalize adjacency: D^{-1/2} * A * D^{-1/2}
        adj_norm = degree_inv_sqrt * adj * degree_inv_sqrt.transpose(0, 1)

        # GCN operation
        x = torch.mm(adj_norm, x)            # (N, d_in)
        x = self.linear(x)                   # (N, d_out)
        return x

class SimpleGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features):
        super(SimpleGCN, self).__init__()
        self.gcn1 = SimpleGCNLayer(in_features, hidden_features)
        self.gcn2 = SimpleGCNLayer(hidden_features, out_features)

    def forward(self, x, adj):
        x = self.gcn1(x, adj)
        x = F.relu(x)
        x = self.gcn2(x, adj)
        return x

### 4.2 Example: Node Classification (Toy Data)

We’ll define a small graph of 5 nodes, construct a toy adjacency matrix, and attempt a simple classification (2 classes). In reality, you’d apply this to a bigger dataset, but the process is the same.

In [3]:
# Toy data
N = 5              # number of nodes
d_in = 4           # dimensionality of input features
x_toy = torch.rand(N, d_in)

# Toy adjacency matrix: connect nodes in a chain + self-loops
adj_toy = torch.zeros(N, N)
for i in range(N - 1):
    adj_toy[i, i+1] = 1
    adj_toy[i+1, i] = 1
# Add self-loops
for i in range(N):
    adj_toy[i, i] = 1

# Labels for each node (0 or 1)
labels = torch.tensor([0, 1, 1, 0, 1])

# Create model
model = SimpleGCN(in_features=d_in, hidden_features=8, out_features=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(50):
    optimizer.zero_grad()
    logits = model(x_toy, adj_toy)  # shape (N, 2)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Final logits
print("\nFinal logits:\n", logits)

Epoch 10, Loss: 0.7042
Epoch 20, Loss: 0.6966
Epoch 30, Loss: 0.6937
Epoch 40, Loss: 0.6922
Epoch 50, Loss: 0.6912

Final logits:
 tensor([[-0.0852, -0.0619],
        [-0.0840, -0.0616],
        [-0.0836, -0.0615],
        [-0.0834, -0.0614],
        [-0.0833, -0.0614]], grad_fn=<AddmmBackward0>)


Not very interesting for just 5 nodes, but the pipeline is the same for bigger graphs! :)

# 5. Relational Graph Convolutional Network (R-GCN)

## 5.1 Why R-GCN?

A typical GCN assumes a single type of relationship between nodes (i.e., all edges are the same). However, in many real-world graphs, edges have different types or relations (e.g., in knowledge graphs, we have different predicates). **Relational GCN** (R-GCN) handles multiple relation types by using separate parameters for each relation.

## 5.2 Formulation

For each node $i$, an R-GCN layer is defined as:

$$
h_i^{(k+1)} = \sigma\left( \sum_{r \in \mathcal{R}} \sum_{j \in \mathcal{N}_r(i)} \frac{1}{c_{i,r}} W_r^{(k)} h_j^{(k)} + W_0^{(k)} h_i^{(k)} \right),
$$

where:

- $\mathcal{R}$ is the set of relation types,
- $W_r^{(k)}$ is a weight matrix for relation $r$,
- $\mathcal{N}_r(i)$ is the set of neighbors of node $i$ under relation $r$,
- $c_{i,r}$ is a normalization constant (e.g., the size of $\mathcal{N}_r(i)$),
- $\sigma$ is a non-linear activation function (e.g., ReLU).


### 5.3 Minimal R-GCN Implementation
Below, we’ll implement a toy version of R-GCN with two relations. We store the adjacency info in a dictionary: `{relation_id: adjacency_matrix}`. Then we sum the contributions from each relation.

In [4]:
class RGCNLayer(nn.Module):
    def __init__(self, in_features, out_features, num_relations):
        super(RGCNLayer, self).__init__()
        self.num_relations = num_relations
        # Each relation has its own linear transform
        self.weight = nn.ModuleList([
            nn.Linear(in_features, out_features, bias=False)
            for _ in range(num_relations)
        ])
        # Relation-independent transform (optional)
        self.self_weight = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x, adjacency_dict):
        """
        x: (N, d_in)
        adjacency_dict: {r: (N,N) adjacency matrix for relation r}
        """
        out = torch.zeros(x.size(0), self.weight[0].out_features)
        
        for r in range(self.num_relations):
            adj_r = adjacency_dict[r]
            # possible normalization per relation
            degree_r = torch.sum(adj_r, dim=1, keepdim=True)
            degree_inv_r = 1.0 / (degree_r + 1e-6)
            agg_r = torch.mm(adj_r, x) * degree_inv_r  # aggregator
            out += self.weight[r](agg_r)

        # Add self-connection
        out += self.self_weight(x)

        return out

class RGCN(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, num_relations):
        super(RGCN, self).__init__()
        self.rgcn1 = RGCNLayer(in_features, hidden_features, num_relations)
        self.rgcn2 = RGCNLayer(hidden_features, out_features, num_relations)

    def forward(self, x, adjacency_dict):
        x = self.rgcn1(x, adjacency_dict)
        x = F.relu(x)
        x = self.rgcn2(x, adjacency_dict)
        return x

### 5.4 Example: Toy Multi-Relation Graph

Let’s say we have 5 nodes and 2 different relations, each with its own adjacency matrix.

In [5]:
# Create random features
x_toy_rgcn = torch.rand(N, d_in)

# Build adjacency for 2 relations
adj_rel0 = torch.zeros(N, N)
adj_rel1 = torch.zeros(N, N)

# Relation 0: connect i -> i+1 mod 5
for i in range(N):
    adj_rel0[i, (i+1) % N] = 1

# Relation 1: connect i -> i+2 mod 5
for i in range(N):
    adj_rel1[i, (i+2) % N] = 1

adj_dict = {
    0: adj_rel0,
    1: adj_rel1
}

# Instantiate RGCN
model_rgcn = RGCN(in_features=d_in, hidden_features=8, out_features=3, num_relations=2)
logits_rgcn = model_rgcn(x_toy_rgcn, adj_dict)
print("R-GCN output logits:\n", logits_rgcn)

R-GCN output logits:
 tensor([[ 0.0488,  0.1305, -0.0340],
        [ 0.0460,  0.1256, -0.0373],
        [ 0.0484,  0.1276, -0.0355],
        [ 0.0463,  0.1254, -0.0371],
        [ 0.0477,  0.1271, -0.0362]], grad_fn=<AddBackward0>)


You could then define labels for these nodes and train similarly with a chosen loss function. Each relation type will learn its own transform.

# 6. Conclusion & Further Reading

In this tutorial, we covered:
- **Graph Notation & Message Passing**
- **Graph Convolutional Network (GCN)** theory + example
- **Relational GCN (R-GCN)** for multi-relational data

## Further Reading
- Kipf & Welling (2017) [Semi-Supervised Classification with GCNs](https://arxiv.org/abs/1609.02907)
- Schlichtkrull et al. (2018) [Modeling Relational Data with R-GCNs](https://arxiv.org/abs/1703.06103)
- Velickovic et al. (2018) [Graph Attention Networks](https://arxiv.org/abs/1710.10903)
- Hamilton et al. (2017) [GraphSAGE](https://arxiv.org/abs/1706.02216)
- [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/)
- [DGL Documentation](https://www.dgl.ai/)

# 7. Advanced Topics

Some popular extensions to GCN/R-GCN include:
1. **Graph Attention Networks (GAT)**: Use attention to weight neighbors differently.
2. **GraphSAGE**: Sample fixed-size neighborhoods for large-scale graphs.
3. **Heterogeneous Graphs**: Different node types and different edge types.
4. **Scaling GNNs**: Efficient training on huge graphs (mini-batching, sampling, etc.).